In [1]:
import pandas as pd
topk = 5

In [2]:
roles_path = '../data/datasets/kenneth_lay/enron_roles.txt'
roles_dict = {}
roles_df = pd.read_csv(roles_path,sep=",")
roles_df.columns = ['emailid','name','position']
print(roles_df.tail())

FileNotFoundError: [Errno 2] No such file or directory: '../data/datasets/kenneth_lay/enron_roles.txt'

In [48]:
hgname = '../data/datasets/real/klay_s.hyp'
# hgname='../data/datasets/real/meetup.hyp'
hyperedges = []
vertex_set = set()
with open(hgname,'r') as f:
    for line in f.readlines():
        edge = line.split(',')
        edge = [e.strip() for e in edge]
        hyperedges.append(edge)
        for v in edge:
            vertex_set.add(v)

In [50]:
# df = pd.read_csv('../data/datasets/kenneth_lay/klay_s.csv',sep=',',engine = 'python')
df = pd.read_csv('../../Hypergraph codes/Cpp-Coredecomp/data/datasets/kenneth_lay/klay_s.csv',sep=',',engine='python')
print(df.head())
print(len(hyperedges))

  Unnamed: 0                                     Message-ID  \
0          0  <19937858.1075858631491.JavaMail.evans@thyme>   
1          1  <20682410.1075859177338.JavaMail.evans@thyme>   
2          2  <25469111.1075840337794.JavaMail.evans@thyme>   
3          4  <31732110.1075849814292.JavaMail.evans@thyme>   
4          5   <5072789.1075855783561.JavaMail.evans@thyme>   

                        Date  \
0  2001-10-18 11:52:23-07:00   
1  2001-07-14 12:33:04-07:00   
2  2001-08-23 11:24:25-07:00   
3  2001-01-03 10:58:00-08:00   
4  2000-08-25 08:26:00-07:00   

                                             Subject  \
0  Quarterly Managing Director Meeting - Monday, ...   
1  FW: Iris Mack's involvement in the Enron Team ...   
2                          Associate/Analyst Program   
3                             1/2/01 Preliminary DPR   
4                                      DPR - 8/24/00   

                                            X-Folder    X-Origin  \
0  \PALLEN (Non-Privile

In [51]:
print('#vertex: ',len(vertex_set),' #edge: ',len(hyperedges))

#vertex:  4718  #edge:  1190


In [54]:
import pickle

def save_dict(a, fname = 'tmp.pickle'):
    with open(fname, 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_dict(fname = 'tmp.pickle'):
    with open(fname, 'rb') as handle:
        b = pickle.load(handle)
        return b
    
def retrieve_email(integer_id,mapN='../data/datasets/kenneth_lay/klay_idtoemail.map'):
    _map = load_dict(mapN)
    if str(integer_id) not in _map:
        return None
    return _map[str(integer_id)]

def retrieve_list_emails(lst_integer_id,mapN='../data/datasets/kenneth_lay/klay_idtoemail.map'):
    _map = load_dict(mapN)
    # print(_map)
    _tmp = []
    for integer_id in lst_integer_id:
        if str(integer_id) not in _map:
            return None
        else:
            _tmp.append(_map[str(integer_id)])
            # print(_tmp)
    return _tmp

def retrieve_stronglyinduced_emails(lst_integer_id, edgeList, df):
    subhyp = []
    for i,value in enumerate(edgeList):
        flag = True
        for u in value:
            if u not in lst_integer_id:
                flag = False
                break
        if flag:
            subhyp.append(i)
        
    return subhyp, df.filter(items = subhyp, axis=0)

# def retrieve_weeklyinduced_complexes(lst_integer_id,human_hg):
#     protein_complex_ids = []
#     for key,value in human_hg.items():
#         flag = True
#         for u in value:
#             if u in lst_integer_id:
#                 protein_complex_ids.append(key)
#                 break 
#     return protein_complex_ids
def load_hg_ascsv(name):
    _list = []
    with open(name,'r') as f:
        for line in f:
            e = [i for i in line.strip().split(',')]
            _list.append(e)
    return _list

def query_position(email):
    em_id = email.split('@')[0]
    _tmp = roles_df[roles_df.emailid == em_id]
    if len(_tmp) == 0:
        return None
    else:
        return roles_df[roles_df.emailid == em_id].position.values[0]
def query_name(email):
    em_id = email.split('@')[0]
    _tmp = roles_df[roles_df.emailid == em_id]
    if len(_tmp) == 0:
        return None
    else:
        return roles_df[roles_df.emailid == em_id].name.values[0]

Densest Subgraph Analysis
------

In [59]:
dataset = 'klay'

deg_d = '../output/density/'+dataset+'_deg.csv'
vol_d = '../output/density/'+dataset+'_nbr.csv'
ck_d = '../output/density/'+dataset+'_clique.csv'

deg_dhg = load_hg_ascsv(deg_d)
vol_dhg = load_hg_ascsv(vol_d)
ck_dhg = load_hg_ascsv(ck_d)
nodes_degD = set()
for e in deg_dhg:
    for u in e:
        nodes_degD.add(u)

nodes_volD = set()
for e in vol_dhg:
    for u in e:
        nodes_volD.add(u)

nodes_ckD = set()
for e in ck_dhg:
    for u in e:
        nodes_ckD.add(u)
        
print(len(nodes_degD),' ',len(nodes_volD),' ',len(nodes_ckD))
print(len(deg_dhg),' ',len(vol_dhg),' ',len(ck_dhg))
# print(nodes_degD)
coreprotein_names = retrieve_list_emails(nodes_degD,mapN='../data/datasets/kenneth_lay/'+dataset+'_idtoemail.map')
# print(coreprotein_names)
sub_complexes_degD,deg_df = retrieve_stronglyinduced_emails(nodes_degD, hyperedges,df)
print('degree-Densest sub-protein complexes: ')
print(deg_df['Subject'].head())
coreprotein_names = retrieve_list_emails(nodes_volD,mapN='../data/datasets/kenneth_lay/'+dataset+'_idtoemail.map')
# print(coreprotein_names)
sub_complexes_volD,vol_df = retrieve_stronglyinduced_emails(nodes_volD, hyperedges,df)
print('volume-Densest sub-protein complexes: ')
print(vol_df['Subject'].head())
print(len(sub_complexes_volD))
coreprotein_names = retrieve_list_emails(nodes_ckD,mapN='../data/datasets/kenneth_lay/'+dataset+'_idtoemail.map')
# print(coreprotein_names)
sub_complexes_ckD,ck_df = retrieve_stronglyinduced_emails(nodes_ckD, hyperedges,df)
print('clique-Densest sub-protein complexes: ')
print(ck_df['Subject'].head())
print(len(sub_complexes_ckD))

166   1949   1949
202   122   907396
degree-Densest sub-protein complexes: 
6      EnronOnline Executive Summary for April 20, 2001
7     EnronOnline Executive Summary for December 12,...
8        EnronOnline Executive Summary for May 31, 2001
12                     NEW POLL FOR MC EXTENDED MEETING
13                 New Poll for the MC Extended Meeting
Name: Subject, dtype: object
volume-Densest sub-protein complexes: 
2                        Associate/Analyst Program
19                     Center for Houston's Future
24    Project Southwood Meeting  - Location Change
95                                          UC/CSU
97                      III Summit of the Americas
Name: Subject, dtype: object
122
clique-Densest sub-protein complexes: 
2                        Associate/Analyst Program
19                     Center for Houston's Future
24    Project Southwood Meeting  - Location Change
95                                          UC/CSU
97                      III Summit of the Ame

In [6]:
import numpy as np
import pickle
M = '../data/datasets/real/meetup_eidtoevent.map'
dataset = 'meetup'

deg_d = '../output/density/'+dataset+'_deg.csv'
vol_d = '../output/density/'+dataset+'_nbr.csv'
ck_d = '../output/density/'+dataset+'_clique.csv'

efile_degD = '../output/density/'+dataset+'_deg_e.csv'
efile_volD = '../output/density/'+dataset+'_nbr_e.csv'
efile_ckD = '../output/density/'+dataset+'_clique_e.csv'

def load_dict(fname = 'tmp.pickle'):
    with open(fname, 'rb') as handle:
        b = pickle.load(handle)
        return b

def load_hg_ascsv(name):
    _list = []
    with open(name,'r') as f:
        for line in f:
            e = [i for i in line.strip().split(',')]
            _list.append(e)
    return _list

def retrieve_list_events(densest_subeFile, mapN='../data/datasets/real/meetup_eidtoevent.map'):
    _map = load_dict(mapN)
    densest_subedges = []
    with open(densest_subeFile,'r') as f:
        for line in f:
            densest_subedges.append(int(line.strip()))
    # print(_map)
    _tmp = []
    for integer_id in densest_subedges:
        if str(integer_id) not in _map:
            return None
        else:
            _tmp.append(_map[str(integer_id)][1])
            # print(_tmp)
    return _tmp
def retrieve_list_eventsCK(ck_dhg,mapN,hypfile):
    # Only for clique
    V_ck = set()
    for e in ck_dhg:
        for u in e:
            V_ck.add(u)
    densest_subedges = []
    projected_hyp = []
    with open(hypfile,'r') as hf:
        count = 0
        for line in hf:
            l = line.strip().split(",")
            s = set(l)
            if s.issubset(V_ck):
                densest_subedges.append(count)
                projected_hyp.append(l)
            count+=1
    _map = load_dict(mapN)
    _tmp = []
    for integer_id in densest_subedges:
        if str(integer_id) not in _map:
            return None
        else:
            _tmp.append(_map[str(integer_id)][1])
            # print(_tmp)
    return projected_hyp, _tmp

def get_volume_density(node_subset, dens_subhyp):
    # Compute nbrs for all the nodes.
    nbrs = {u: set() for u in node_subset}
    for hyperedge in dens_subhyp:
        absE = len(hyperedge)-1
        for u in hyperedge:
            for v in hyperedge:
                if u!=v:
                    nbrs[u].add(v)
    print('average #participants: ',np.mean([len(hyperedge) for hyperedge in dens_subhyp]))
    return sum([len(nbrs[u]) for u in node_subset])/len(node_subset)

deg_dhg = load_hg_ascsv(deg_d)
vol_dhg = load_hg_ascsv(vol_d)
ck_dhg = load_hg_ascsv(ck_d)
nodes_degD = set()
for e in deg_dhg:
    for u in e:
        nodes_degD.add(u)

nodes_volD = set()
for e in vol_dhg:
    for u in e:
        nodes_volD.add(u)

nodes_ckD = set()
for e in ck_dhg:
    for u in e:
        nodes_ckD.add(u)
        

# print(nodes_degD)
print('degree-Densest sub-hyp Events: ')
coreprotein_names = retrieve_list_events(efile_degD,mapN=M)
print(coreprotein_names)
print('vol. density: ',get_volume_density(nodes_degD, deg_dhg))
# sub_complexes_degD,deg_df = retrieve_stronglyinduced_emails(nodes_degD, hyperedges,df)
# # print(deg_df['Subject'].head())
print('volume-Densest sub-hyp Events: ')
coreprotein_names = retrieve_list_events(efile_volD,mapN=M)
print(coreprotein_names)
print('volume-density: ',get_volume_density(nodes_volD, vol_dhg))
# sub_complexes_volD,vol_df = retrieve_stronglyinduced_emails(nodes_volD, hyperedges,df)
# # print(vol_df['Subject'].head())
# # print(len(sub_complexes_volD))
print('clique-Densest sub-hyp Events: ')
projected_hyp, coreprotein_names_ck = retrieve_list_eventsCK(ck_dhg,mapN=M,hypfile = '../data/datasets/real/meetup.hyp') # Take union of list_of_edges in ck to find the nodes. Read hyp file
print(coreprotein_names_ck)
print('volume-density: ',get_volume_density(nodes_ckD, projected_hyp),' degree-density as clique: ',len(ck_dhg)/len(nodes_ckD))
# sub_complexes_ckD,ck_df = retrieve_stronglyinduced_emails(nodes_ckD, hyperedges,df)
# print('clique-Densest sub-protein complexes: ')
# # print(ck_df['Subject'].head())
# # print(len(sub_complexes_ckD))

print('|V*_deg|=',len(nodes_degD),' |V*_vol|=',len(nodes_volD),' |V*_clique|=',len(nodes_ckD))
print('|E*_deg|=',len(deg_dhg),' |E*_vol|=',len(vol_dhg),' |E*_clique|=',len(coreprotein_names_ck))

degree-Densest sub-hyp Events: 
['Trivia Night at Plantation Pub', 'Trivia Night at Plantation Pub', "1st Tuesday Ladies Lunch at Chili's Bellevue", 'Wednesday night dinner at Voodoo Gumbo', 'Tuesday night dinner at {Pub}licity', 'Dinner At KYOTO', "Dinner At Dalton's Grill", 'Dinner At Plantation Pub Formerly Three Stones', 'Dinner at Lucky Bamboo China Bistro', "Beginner's Hike at Edwin Warner, Sun. 6:30", 'Dinner at Lemongrass Thai and Sushi', "Trivia Night at Nacho's Mexican Restaurant HWY 100", "Trivia Night at Nacho's Mexican Restaurant HWY 100", "Trivia Night at Nacho's Mexican Restaurant HWY 100", 'Ladies Lunch Thursday June 9', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Trivia Night at Three Stones Pub', 'Beginners Hike at Edwin Warner Jan. 3', "Celebrate New Year's Eve with Friends", 'Last-Minute Dinner T

In [ ]:
1. Show our vol dens > Projected-back clique.
2. Projection er jonno. Loss  korchi.

In [82]:
# M = '../data/datasets/real/meetup_eidtoevent.map'
# dataset = 'meetup'

# deg_d = '../output/density/'+dataset+'_deg.csv'
# vol_d = '../output/density/'+dataset+'_nbr.csv'
# ck_d = '../output/density/'+dataset+'_clique.csv'

# efile_degD = '../output/density/'+dataset+'_deg_e.csv'
# efile_volD = '../output/density/'+dataset+'_nbr_e.csv'
# efile_ckD = '../output/density/'+dataset+'_clique_e.csv'

# def retrieve_list_events(densest_subeFile, mapN='../data/datasets/real/meetup_eidtoevent.map'):
#     _map = load_dict(mapN)
#     densest_subedges = []
#     with open(densest_subeFile,'r') as f:
#         for line in f:
#             densest_subedges.append(int(line.strip()))
#     # print(_map)
#     _tmp = []
#     for integer_id in densest_subedges:
#         if str(integer_id) not in _map:
#             return None
#         else:
#             _tmp.append(_map[str(integer_id)][1])
#             # print(_tmp)
#     return _tmp
# def retrieve_list_eventsCK(ck_dhg,mapN,hypfile):
#     # Only for clique
#     V_ck = set()
#     for e in ck_dhg:
#         for u in e:
#             V_ck.add(u)
#     densest_subedges = []
#     with open(hypfile,'r') as hf:
#         count = 0
#         for line in hf:
#             l = line.strip().split(",")
#             s = set(l)
#             if s.issubset(V_ck):
#                 densest_subedges.append(count)
#             count+=1
#     _map = load_dict(mapN)
#     _tmp = []
#     for integer_id in densest_subedges:
#         if str(integer_id) not in _map:
#             return None
#         else:
#             _tmp.append(_map[str(integer_id)][1])
#             # print(_tmp)
#     return list(V_ck), _tmp


# deg_dhg = load_hg_ascsv(deg_d)
# vol_dhg = load_hg_ascsv(vol_d)
# ck_dhg = load_hg_ascsv(ck_d)
# nodes_degD = set()
# for e in deg_dhg:
#     for u in e:
#         nodes_degD.add(u)

# nodes_volD = set()
# for e in vol_dhg:
#     for u in e:
#         nodes_volD.add(u)

# nodes_ckD = set()
# for e in ck_dhg:
#     for u in e:
#         nodes_ckD.add(u)
    
# # print(nodes_degD)
# print('degree-Densest sub-hyp Events: ')
# coreprotein_names = retrieve_list_events(efile_degD,mapN=M)
# print(coreprotein_names)
# # sub_complexes_degD,deg_df = retrieve_stronglyinduced_emails(nodes_degD, hyperedges,df)
# # # print(deg_df['Subject'].head())
# print('volume-Densest sub-hyp Events: ')
# coreprotein_names = retrieve_list_events(efile_volD,mapN=M)
# print(coreprotein_names)
# # sub_complexes_volD,vol_df = retrieve_stronglyinduced_emails(nodes_volD, hyperedges,df)
# # # print(vol_df['Subject'].head())
# # # print(len(sub_complexes_volD))
# print('clique-Densest sub-hyp Events: ')
# nodes_ckD, coreprotein_names_ck = retrieve_list_eventsCK(ck_dhg,mapN=M,hypfile = '../data/datasets/real/meetup.hyp') # Take union of list_of_edges in ck to find the nodes. Read hyp file
# print(coreprotein_names_ck)
# # sub_complexes_ckD,ck_df = retrieve_stronglyinduced_emails(nodes_ckD, hyperedges,df)
# # print('clique-Densest sub-protein complexes: ')
# # # print(ck_df['Subject'].head())
# # # print(len(sub_complexes_ckD))

# print(len(nodes_degD),' ',len(nodes_volD),' ',len(nodes_ckD))
# print(len(deg_dhg),' ',len(vol_dhg),' ',len(coreprotein_names_ck))

In [62]:
# load_dict(M)

In [24]:
# Hypergraph Class
import math
import itertools
import random
class Hypergraph:
    """ 
    Our own hypergraph representation class. 
    We store hyperedge list in compressed format using two things- 1) e_indices (a dict) 2) e_nodes (a list)
    Although edge-centric queries (e.g. edge enumeration) are facilitated in this way, node-centric queries are not convenient.
    To support node-centric queries, we also maintain incidence dictionary inc_dict (key = v_ids, values = incident edge ids)
    """

    def __init__(self, _edgedict=None):
        
        self.e_indices = {}  # (position, position+edge_size) of edge e in e_nodes list
        self.e_nodes = []  # flattened edge list
        self.inc_dict = {}  # key: nodeid, value = ids of incident edges (set)
        # degree pre-compute => degree_dict or len_incedge = {}
        self.degree_dict = {}
        self.init_nbrsize = {} # initial nbrhood sizes. can be precomputed.
        self.init_nbr = {}
        self.init_eids = {}
        self.init_nodes = []
        if _edgedict is None or len(_edgedict)==0:  # Returns an empty Hypergraph
            return

        self.i = 0
        for e_id, e in _edgedict.items():
            _len = len(e)
            
            self.e_indices[e_id] = (self.i, self.i + _len)
            self.init_eids[e_id] = (self.i, self.i + _len)
            for v in e:
                self.e_nodes.append(v)
                if v not in self.inc_dict:
                    self.inc_dict[v] = set()  # create incident edge entry for v
                    self.init_nodes.append(v)
                self.inc_dict[v].add(e_id)  # incident edge update
                self.degree_dict[v] = self.degree_dict.get(v, 0) + 1  # degree update
                nbr_v = self.init_nbr.get(v, set()).union(e)
                nbr_v.remove(v)
                self.init_nbrsize[v] = len(nbr_v)  # neighbourhood length update
                self.init_nbr[v] = nbr_v  # neighbourbood set update
            self.i += _len

        self.init_nodes = sorted(self.init_nodes)
    def get_topk_degNodes(self,k):
        """ Returns top-k highest degree nodes """
        import heapq
        degree_loftup = list(self.degree_dict.items())
        k_keys_sorted = heapq.nlargest(k, degree_loftup,lambda e: e[1])
        return k_keys_sorted

    def get_topk_nbrNodes(self,k):
        """ Returns top-k highest degree nodes """
        import heapq
        nbr_loftup = list(self.init_nbrsize.items())
        k_keys_sorted = heapq.nlargest(k, nbr_loftup,lambda e: e[1])
        return k_keys_sorted

    def get_topk_hyperedges(self,k):
        """ Returns top-k largest hyperedges """
        import heapq 

        _tmp = []
        for eid in self.e_indices:
            _tmp.append((eid,self.e_indices[eid][1] - self.e_indices[eid][0]))
        return heapq.nlargest(k, _tmp,lambda e: e[1])

    def __len__(self):
        return len(self.e_indices)
def get_topk_listoftuples(k,listoftups):
    import heapq
    k_keys_sorted = heapq.nlargest(k, listoftups,lambda tup: tup[1])
    return k_keys_sorted

In [25]:
def retrieve_list_emails(l):
    
vol_dhg = Hypergraph({i: hyperedges[i] for i in sub_complexes_volD})
# print(len(vol_dhg.init_nodes),' ',len(vol_dhg))
voldhg_topk = vol_dhg.get_topk_nbrNodes(topk)
degdhg_topk = vol_dhg.get_topk_degNodes(topk)

topk_nbr_emails= retrieve_list_emails([i[0] for i in voldhg_topk])
topk_deg_emails = retrieve_list_emails([i[0] for i in degdhg_topk])
# print(topk_nbr_emails)


print('top-k degree nodes in the  vol-densest sub-hyp: ')
# print([(i,query_position(i)) for i in topk_deg_emails])
print([(i,query_name(i),query_position(i)) for i in topk_deg_emails])
# print([vol_dhg.degree_dict[i[0]] for i in degdhg_topk])

print('top-k nbr nodes in the  vol-densest sub-hyp: ')
print([(i,query_name(i),query_position(i)) for i in topk_nbr_emails])
# print([vol_dhg.init_nbrsize[i[0]] for i in voldhg_topk])
print('----------')
deg_dhg = Hypergraph({i: hyperedges[i] for i in sub_complexes_degD})
# print(len(deg_dhg.init_nodes),' ',len(deg_dhg))
# print(len(deg_dhg))
degdhg_topk =deg_dhg.get_topk_degNodes(topk)
voldhg_topk = deg_dhg.get_topk_nbrNodes(topk)
topk_nbr_emails= retrieve_list_emails([i[0] for i in voldhg_topk])
topk_deg_emails = retrieve_list_emails([i[0] for i in degdhg_topk])
print('top-k degree nodes in the  degree-densest sub-hyp: ')
print([(i,query_name(i),query_position(i)) for i in topk_deg_emails])
# print([deg_dhg.degree_dict[i[0]] for i in degdhg_topk])
print('top-k nbr nodes in the  degree-densest sub-hyp: ')
print([(i,query_name(i),query_position(i)) for i in topk_nbr_emails])
# print([deg_dhg.init_nbrsize[i[0]] for i in voldhg_topk])
# print(topk_deg_emails)
# print([deg_dhg.degree_dict[u] for u in deg_dhg.get_topk_degNodes(topk)])
print('---------------')
ck_dhg = Hypergraph({i: hyperedges[i] for i in sub_complexes_ckD})
degdhg_topk =ck_dhg.get_topk_degNodes(topk)
voldhg_topk = ck_dhg.get_topk_nbrNodes(topk)
topk_nbr_emails= retrieve_list_emails([i[0] for i in voldhg_topk])
topk_deg_emails = retrieve_list_emails([i[0] for i in degdhg_topk])
print('top-k degree nodes in the clique-densest sub-hyp: ')
print([(i,query_name(i),query_position(i)) for i in topk_deg_emails])
print('top-k nbr nodes in the  clique-densest sub-hyp: ')
print([(i,query_name(i),query_position(i)) for i in topk_nbr_emails])

top-k degree nodes in the  vol-densest sub-hyp: 
[('kenneth.lay@enron.com', 'Kenneth Lay', 'CEO'), ('greg.whalley@enron.com', 'Greg Whalley', 'President'), ('jeff.skilling@enron.com', 'Jeffery Skilling', 'CEO'), ('mark.frevert@enron.com', 'Mark A. Frevert', 'Chairman and CEO(Enron WholeSale Services)'), ('mark.koenig@enron.com', 'Mark Koenig', 'Head(Investor Relations)')]
top-k nbr nodes in the  vol-densest sub-hyp: 
[('greg.martin@enron.com', 'Gregory Martin', 'Analyst'), ('dustin.collins@enron.com', 'Dustin Collins', 'Associate(Enron Global Commodities)'), ('andrea.richards@enron.com', None, None), ('sladana-anna.kulic@enron.com', None, None), ('maureen.mcvicker@enron.com', 'Maureen McVicker', 'Assistant(Steven Kean)')]
----------
top-k degree nodes in the  degree-densest sub-hyp: 
[('kenneth.lay@enron.com', 'Kenneth Lay', 'CEO'), ('greg.whalley@enron.com', 'Greg Whalley', 'President'), ('mark.koenig@enron.com', 'Mark Koenig', 'Head(Investor Relations)'), ('jeffrey.mcmahon@enron.com'